In [89]:
from xml.dom import minidom
import numpy as np
import geopy.distance

In [63]:
def chunk_nodes(nodes):
    list1 = list(nodes)[:-1]
    list2 = list(nodes)[1:]
    
    return list(zip(list1, list2))

In [23]:
dom = minidom.parse("interpreter")

In [72]:
ways = dom.getElementsByTagName('way')
nodes_tags = dom.getElementsByTagName('node')
nodes = {}
edges = {}

In [97]:
def calculate_distance(node1, node2):
    return geopy.distance.vincenty(nodes[node1], nodes[node2]).m

In [86]:
for n in nodes_tags:
    nodes[int(n.getAttribute('id'))] = (float(n.getAttribute('lat')), float(n.getAttribute('lon')))

In [102]:
highest = 0
counter = 1
for w in ways:
    tags = w.getElementsByTagName('tag')
    width = 2
    maxspeed = 0
    oneway = 1
    
    for t in tags:
        key = t.getAttribute('k')
        value = t.getAttribute('v')
        
        if (key == 'width'):
            width = float(value)
        elif (key == 'maxspeed'):
            maxspeed = int(value)/3.6
        elif (key == 'oneway'):
            oneway = 1 if (value == 'yes') else 0
    
    chunks = chunk_nodes(w.getElementsByTagName('nd'))
    for chunk in chunks:
        n1 = chunk[0].getAttribute('ref')
        n2 = chunk[1].getAttribute('ref')
        
        length = calculate_distance(int(n1), int(n2))
        edge = {
            'freespeed': maxspeed,
            'length': length,
            'capacity': width*length,
            'from': int(n1),
            'to': int(n2),
            'oneway': oneway,
            'modes': 'car',
            'origid': int(w.getAttribute('id')),
            'type': 'residential'
        }
        edges[counter] = edge
        counter += 1

In [136]:
import lxml.etree
import lxml.builder
from lxml import etree as ET

root_et = ET.Element('network')
nodes_et = ET.SubElement(root_et, 'nodes')
links_et = ET.SubElement(root_et, 'links')

for node_id, attrs in nodes.items():
    ET.SubElement(nodes_et, 'node', id=str(node_id), x=str(attrs[0]), y=str(attrs[1]))


for link_id, attrs in edges.items():
    ele = ET.SubElement(links_et, 'link',
                    id=str(link_id),
                  capacity=str(attrs["capacity"]),
                  freespeed=str(attrs["freespeed"]),
                  to=str(attrs["to"]),
                  length=str(attrs["length"]),
                  modes=str(attrs["modes"]),
                  oneway=str(attrs["oneway"]),
                    origid=str(attrs["origid"]),
                type=str(attrs["type"]))
    ele.set('from', str(attrs["from"]))
    
  
tree = ET.ElementTree(root_et)
tree.write('output.xml', pretty_print=True, xml_declaration=True,   encoding="utf-8")